In [ ]:
import json
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import datetime, date, timedelta
from pytz import timezone

def lambda_handler(event, context):
    schema="Gans"   # name of the database you want to use here
    host="wbs-project3-db.cyftikg4p0dk.us-east-1.rds.amazonaws.com"        # to connect to your local server
    user="admin"
    password="Swathy11" # your password!!!!
    port=3306
    con=f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    tomorrows_flight_arrivals(con)
    #data = [[1, 'EDDB', 'Berlin Brandenburg Airport'], [2, 'EDDH', 'Hamburg Helmut Schmidt Airport'], [3, 'EGLL','London Heathrow Airport'], [4, 'EGCC','Manchester Airport'],[5, 'LEBL','Josep Tarradellas Barcelona-El Prat Airport']]
    #codes= pd.DataFrame(data, columns=['city_id', 'icao', 'airport_name'])
    #codes.to_sql('airports',if_exists='append',con=con,index=False)
    #weather(con)
    #recreate_wiki(con)
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

'''def recreate_wiki(con):
 city= 'Barcelona'
 list_for_df= []
 url = f'https://en.wikipedia.org/wiki/{city}'
 r = requests.get(url)
 soup = BeautifulSoup(r.content, 'html.parser')

    # here we initialise our empty dictionary for the city
 response_dict = {}

    # here we fill the dictionary with information using the ids, classes, and selectors that we found in the html
 response_dict['city'] = soup.select(".firstHeading")[0].get_text()
 response_dict['country'] = soup.select(".infobox-data")[0].get_text()
 response_dict['latitude'] = soup.select(".latitude")[0].get_text()
 response_dict['longitude'] = soup.select(".longitude")[0].get_text()
 response_dict['population'] = soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
    # add our dictionary for the city to list_for_df
 list_for_df.append(response_dict)
  
  # make the DataFrame
 cities_df = pd.DataFrame(list_for_df)

  # fixing latitude
 cities_df['latitude'] = cities_df['latitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
  # fixing longitude
 cities_df['longitude'] = cities_df['longitude'].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
 cities_df['population']= cities_df['population'].str.replace(',','')
  # fixing elevation
 cities_df.to_sql('towns',if_exists='append',con=con,index=False)
  # return the DataFrame
 return cities_df
'''


def weather(con):
   data = [[1, 'Berlin', 'DE'], [2, 'Hamburg', 'DE'], [3, 'London','GB'], [4, 'Manchester','GB'],[5, 'Barcelona','ES']]
   OWM_KEY='83fa26bdfdf0de553c224eb0faf87136'
   cities= pd.DataFrame(data, columns=['city_id', 'city', 'country_code'])
   url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities['city'][0]},{cities['country_code'][0]}&appid={OWM_KEY}&units=metric"
   response = requests.get(url)
   weather_dict = {'city_id': [],
                'forecast_time': [],
                'outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'wind_speed': [],
                'pop': []}
# iterating over items in 'list' node and selecting the subnodes gives us the exact info we need 
   for hour in response.json()['list']:
      weather_dict['city_id'].append(cities['city_id'][0])
      weather_dict['temperature'].append(hour['main']['temp'])
      weather_dict['wind_speed'].append(hour['wind']['speed'])
      weather_dict['forecast_time'].append(hour['dt_txt'])
      weather_dict['outlook'].append(hour['weather'][0]['description'])
      weather_dict['temperature_feels_like'].append(hour['main']['feels_like'])
      weather_dict['pop'].append(hour['pop'])
      weather_df = pd.DataFrame(weather_dict)
   weather_df.to_sql('weathers',         # 'iss_logs'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False)        # index = False -> will not send index column to database

 
 


def tomorrows_flight_arrivals(con):
  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  tomorrow = (today + timedelta(days=1))
  list_for_df = []
  icao_list=['EDDH']
  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      headers = {
	       "X-RapidAPI-Key": "fede0c7146msh1bb3d4ba98e89a5p1df66ajsn5f7e89b742e5",
	       "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"}

      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['arrival_icao'] = icao
        flights_dict['arrival_time'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['flight_num'] = flight.get('number', None)
        list_for_df.append(flights_dict)
        flight_data=pd.DataFrame(list_for_df)
        flights=flight_data.loc[:,['flight_num','departure_icao','arrival_icao','arrival_time']]
        flights['arrival_time']=pd.to_datetime(flights['arrival_time']).dt.tz_convert(None)
        flights
        flights.to_sql('flights',       
              if_exists='append',
              con=con,          
              index=False)
